In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from pathos.multiprocessing import ProcessingPool
from pathos.multiprocessing import ProcessPool
import os
import progressbar
import pickle

In [2]:
df = pd.read_csv('D:\Google Drive\내 드라이브\LiveCommerse\Total_DataFrame.csv')
df2 = df.sort_values(by='Num_Sub', ascending=False)
df3 = df2[df2['Tour/Experience'] == 1]
df4 = df3[(df3['Food']==0) & (df3['Beauty']==0) & (df3['Life']==0) & (df3['Fashion']==0) & (df3['Kids']==0) & (df3['Tech']==0) & (df3['Hobby/Leisure']==0) & (df3['Culture']==0)]
df4.Num_Video = df4.Num_Video.astype(int)
df5 = df3[(df3['Food']==1) | (df3['Beauty']==1) | (df3['Life']==1) | (df3['Fashion']==1) | (df3['Kids']==1) | (df3['Tech']==1) | (df3['Hobby/Leisure']==1) | (df3['Culture']==1)]


C:\Users\gkfua\AppData\Local\Temp\ipykernel_21444\2036529964.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\Google Drive\내 드라이브\LiveCommerse\Total_DataFrame.csv')
C:\Users\gkfua\AppData\Local\Temp\ipykernel_21444\2036529964.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.Num_Video = df4.Num_Video.astype(int)


In [3]:
tour_df = df4[['Channel_Name', 'Channel_number', 'Num_Video']]
tour_df['Num_Video'] = tour_df['Num_Video'].astype(int)

C:\Users\gkfua\AppData\Local\Temp\ipykernel_21444\3005887218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tour_df['Num_Video'] = tour_df['Num_Video'].astype(int)


In [4]:
chrome_driver_path = 'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe'

class MyCrawler:
    def __init__(self, numprocess, path_save):
        self.n_process = numprocess
        self.pool = ProcessPool(nodes=self.n_process)
        self.path_save = path_save
        
    def multi_processing(self, sites):
        def chat_crawler(url):
            # 사용하는 Library Import
            import pandas as pd
            from selenium import webdriver
            from bs4 import BeautifulSoup
            from selenium.webdriver.common.by import By
            from selenium.webdriver.chrome.options import Options
            from selenium.webdriver.chrome.service import Service
            from webdriver_manager.chrome import ChromeDriverManager
            import time
            import os
            chrome_driver_path = 'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe'
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            # driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)
 
            # 원하는 사이트 로드
            driver.get(url)
            save_number = str(url)
            save_number = save_number[44:50]
            
            # 1분 대기 ... 여러 사이트 열어서 시간이 좀 걸림.
            time.sleep(30)

            # 자동으로 2배속 설정.
            buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
            buttun.click()
            time.sleep(1)
            buttun.click()
            
            # 반복 횟수 설정.
            repeat_n = 24
            empty = []
            df = pd.DataFrame({'name' : empty, 'chat' : empty})
            
            # 채팅 및 아이디 크롤링.
            for re_n in range(repeat_n):
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
            
                chat_list_temp = []
                for i in temp_list:
                    for j in i:
                        for k in j:
                            chat = k.select('span')
                            chat = str(chat)
                            chat_list_temp.append(chat)

                name_list_temp = []
                for i in temp_list:
                    for j in i:
                        name = j.select('strong')
                        name = str(name)
                        if 'nickname' in name:
                            name_list_temp.append(name)

                chat_list = []
                for i in chat_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        chat_list.append(i)
                    except:
                        continue
                    
                name_list = []
                for i in name_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        name_list.append(i)
                    except:
                        continue
                df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
                df = pd.concat([df, df_temp], ignore_index=True)
                df.drop_duplicates(inplace=True)
                df.reset_index(drop=True, inplace=True)
                df.to_csv(self.path_save + '\Video_{}.csv' .format(save_number), index=False)
                time.sleep(200)

            # df 형성 및 저장.
            df.drop_duplicates(inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.to_csv(self.path_save + '\Video_{}.csv' .format(save_number), index=False)
            
            driver.quit()
        
        self.pool.map(chat_crawler, sites)

In [5]:
with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\Tour_only_df\Complete_list.pkl', 'rb') as f:
    Complete_list = pickle.load(f)
    
Complete_list

[90368,
 93318,
 18823,
 76553,
 40207,
 67856,
 68113,
 68114,
 92563,
 69787,
 66849,
 67619,
 67238,
 69543,
 76202,
 63916,
 38573,
 72886,
 67510,
 62139,
 70207,
 70208,
 87749,
 89935,
 92112,
 89047,
 92519,
 71784,
 68969,
 69736,
 93165,
 93297,
 64371,
 67444,
 59387,
 69573,
 65073,
 87583,
 54844,
 93549,
 60996,
 66695,
 91209,
 91780,
 66896,
 56282,
 88592,
 66794,
 79990,
 66601,
 87548,
 69388,
 110358,
 66077,
 91374,
 96130,
 92051,
 78217,
 72281,
 107333,
 100383,
 96367,
 88298,
 74994,
 87868,
 76684,
 66440,
 106771,
 65305,
 89324,
 67477,
 89544,
 67456,
 76033,
 89635,
 89194,
 64823,
 95414,
 92120,
 90496,
 100889,
 96458,
 94093,
 97409,
 98858,
 106544,
 97765]

In [6]:
path = 'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\Tour_only_df'
Complete_list = []
check_list = []

for i in range(len(tour_df)):
    temp = tour_df.iloc[i]
    temp_id = temp[0]
    temp_number = int(temp[1])
    temp_video = temp[2]
    
    if temp_video > 20:
        print(f"Length over, Crwaling Fail. {temp_id}, {temp_number}")
        check_list.append(temp_number)
        with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\Tour_only_df\Check_list.pkl', 'wb') as q:
            pickle.dump(check_list, q)
            
    elif 21 > temp_video:
        with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\Tour_only_df\Complete_list.pkl', 'rb') as f:
            Complete_list = pickle.load(f)
    
        if temp_number not in Complete_list:
            dir_path = path + f'\{temp_id}_{temp_number}'

            chrome_driver_path = 'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe'
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)
            url = f'https://shoppinglive.naver.com/channels/{temp_number}'
            driver.get(url)
            time.sleep(3)
        
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            url_list = soup.select('a.VideoBoxLinkWrapper_wrap_GLkZS')
            
            try:
                os.mkdir(dir_path)
            except:
                print('Dir_already exist')
                
            temp_list = []
            for j in url_list:
                j = str(j)
                j = j.split('href="')[1]
                j = j.split('"><div')[0]
                if 'replays' in j:
                    temp_list.append(j)
            
            n = 5
            slice_list = [temp_list[i*n : n*(i+1)] for i in range(0, (len(temp_list) + n -1)//n)]
            k = len(slice_list)
            
            print(f'Channel {temp_id}_{temp_number} Crawling start. K = {k}')
            
            for p, l in enumerate(slice_list):
                print('k = {} start' .format(p+1))
                try:
                    print('Processing ...')
                    print(l)
                    Crawler = MyCrawler(len(l), dir_path)
                    Crawler.multi_processing(l)
                except:
                    print(f'Fail. Channel = {temp_id}, Number = {temp_number}')
                    print(l)
                    
            Complete_list.append(temp_number)
            with open('D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\Tour_only_df\Complete_list.pkl', 'wb') as f:
                pickle.dump(Complete_list, f)
            print(f'Channel {temp_id}_{temp_number} Crawling End.')
        else:
            print(f'Alread Complete : {temp_number}')

Length over, Crwaling Fail. 네이버 여행상품, 66400
Alread Complete : 67619
Alread Complete : 70207
Alread Complete : 76553
Length over, Crwaling Fail. 올마이투어, 65650
Alread Complete : 67444
Alread Complete : 67238
Alread Complete : 68969
Alread Complete : 92563
Alread Complete : 66849
Alread Complete : 72886
Alread Complete : 63916
Alread Complete : 62139
Alread Complete : 76202
Length over, Crwaling Fail. 여행온여수, 78265
Alread Complete : 64371
Alread Complete : 70208
Alread Complete : 93165
Alread Complete : 92519
Alread Complete : 67510
Alread Complete : 69787
Alread Complete : 89047
Alread Complete : 93318
Alread Complete : 68114
Alread Complete : 66440
Alread Complete : 87749
Alread Complete : 69543
Alread Complete : 92112
Alread Complete : 40207
Alread Complete : 18823
Alread Complete : 67856
Alread Complete : 89935
Alread Complete : 59387
Alread Complete : 68113
Alread Complete : 106771
Alread Complete : 90368
Alread Complete : 93297
Alread Complete : 71784
Alread Complete : 69736
Alread Co

C:\Users\gkfua\AppData\Local\Temp\ipykernel_21444\3108750994.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)


Dir_already exist
Channel 메이즈랜드_92061 Crawling start. K = 1
k = 1 start
Processing ...
['https://view.shoppinglive.naver.com/replays/577878?fm=shoppinglive&amp;sn=home']
Channel 메이즈랜드_92061 Crawling End.
Channel 현대유람선 CRUISE_97167 Crawling start. K = 1
k = 1 start
Processing ...
['https://view.shoppinglive.naver.com/replays/702622?fm=shoppinglive&amp;sn=home']
Channel 현대유람선 CRUISE_97167 Crawling End.
Channel 광안리 오션뷰 에이치에비뉴_60123 Crawling start. K = 1
k = 1 start
Processing ...
['https://view.shoppinglive.naver.com/replays/16869?fm=shoppinglive&amp;sn=home']
Fail. Channel = 광안리 오션뷰 에이치에비뉴, Number = 60123
['https://view.shoppinglive.naver.com/replays/16869?fm=shoppinglive&amp;sn=home']
Channel 광안리 오션뷰 에이치에비뉴_60123 Crawling End.
Alread Complete : 88298
Channel 여의도 메리어트 호텔_67188 Crawling start. K = 1
k = 1 start
Processing ...
['https://view.shoppinglive.naver.com/replays/109681?fm=shoppinglive&amp;sn=home']
Channel 여의도 메리어트 호텔_67188 Crawling End.
Channel 야놀자레저_72890 Crawling start. K = 1
